In [6]:
from openai import OpenAI
import pandas as pd
from dotenv import load_dotenv
import os
from time import sleep

load_dotenv()

True

In [7]:
client = OpenAI(
  api_key=os.getenv("OPENAI_API_KEY"),
)


In [8]:
df = pd.read_csv("2023-01_skynews_articles.csv", sep=';')
print(df.shape)
df.dropna(subset=['Tags'], inplace=True)
df_first = df.drop(df[df['Tags'] == "['N/A']"].index)
df_second = df_first.drop(df_first[df_first['Tags'] == "[]"].index)
print(df.shape)
df_second.to_csv("2023-01_skynews_articles_clean.csv", sep=';')

(3112, 8)
(3112, 8)


In [12]:
df = pd.read_csv("2023-01_skynews_articles_clean_02.csv", sep=';')
df = df.reset_index()
print(df.shape)

(179, 10)


In [13]:
def get_res(batch):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
        {"role": "system", "content": "You will be provided with a batch of items. Each item consists of a title and one or multiple tags that both belong to one news article. The items start with <itemStart> and end with <itemEnd>. Your task is to classify the topic of a news article by looking at its title and tags in either Category-1: politics, Category-2: business and economy, Category-3: environment, Category-4: sports, Category-5: entertainment and culture, Category-6: science and technology or Category-7: health. Return for each article only the number of the category in a python list where the result matches the order of the input."},
        {"role": "user", "content": f"{batch}"}
        ],
        temperature=0,
        max_tokens=1000
    )
    categories = response.choices[0].message.content[1:-1].split(",")
    return categories

In [14]:
batchSize = 10

cat = {
     1: "politics",
     2: "business and economy",
     3: "environment",
     4: "sports",
     5: "entertainment and culture",
     6: "science and technology",
     7: "health"
    }

counter = 0
idx_lo = 0
idx_hi = 0
batch = ""
for index, row in df.iterrows():
    print(index)
    if counter < batchSize:
        title = row['Title']
        tags = row['Tags'][1:-1]
        counter += 1
        batch += f"<itemStart> ARTICLE-{counter}: Title: {title}; Tags: {tags} <itemEnd>"
    else:
        categories = get_res(batch)
        print(categories)
        if len(categories) < 10:
             break
        for idx_category, category in enumerate(categories):
            print(index + 1 - len(categories)+ idx_category, category)
            df.loc[index - len(categories) + idx_category, ['Chat GPT']] = category.strip().replace(",", "")
        sleep(3)
        title = row['Title']
        tags = row['Tags'][1:-1]
        counter = 1
        batch = f"<itemStart> ARTICLE-{counter}: Title: '{title}'; Tags: {tags}] <itemEnd>"

if len(batch) > 0:
    categories = get_res(batch)
    for idx_category, category in enumerate(categories):
            print(df.shape[0] - len(categories)+ idx_category, category)
            df.loc[df.shape[0] - len(categories) + idx_category, ['Chat GPT']] = category.strip().replace(",", "")


df.to_csv("2023_skynews_chatgpt_results_02.csv", sep=';')


0
1
2
3
4
5
6
7
8
9
10
['1', ' 1', ' 5', ' 5', ' 1', ' 6', ' 2', ' 6', ' 2', ' 2']
1 1
2  1
3  5
4  5
5  1
6  6
7  2
8  6
9  2
10  2


/var/folders/sh/fjb1r_5j6gxcy4lzfcc4_4zr0000gn/T/ipykernel_80177/3940162167.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[index - len(categories) + idx_category, ['Chat GPT']] = category.strip().replace(",", "")


11
12
13
14
15
16
17
18
19
20
['1', ' 7', ' 5', ' 1', ' 5', ' 4', ' 7', ' 1', ' 4', ' 7']
11 1
12  7
13  5
14  1
15  5
16  4
17  7
18  1
19  4
20  7
21
22
23
24
25
26
27
28
29
30
['2', ' 4', ' 6', ' 1', ' 2', ' 6', ' 5', ' 5', ' 7', ' 5']
21 2
22  4
23  6
24  1
25  2
26  6
27  5
28  5
29  7
30  5
31
32
33
34
35
36
37
38
39
40
['2', ' 1', ' 7', ' 3', ' 5', ' 5', ' 7', ' 3', ' 4', ' 7']
31 2
32  1
33  7
34  3
35  5
36  5
37  7
38  3
39  4
40  7
41
42
43
44
45
46
47
48
49
50
['1', ' 5', ' 2', ' 2', ' 1', ' 6', ' 7', ' 2', ' 5', ' 1']
41 1
42  5
43  2
44  2
45  1
46  6
47  7
48  2
49  5
50  1
51
52
53
54
55
56
57
58
59
60
['1', ' 2', ' 1', ' 3', ' 4', ' 2', ' 1', ' 3', ' 5', ' 5']
51 1
52  2
53  1
54  3
55  4
56  2
57  1
58  3
59  5
60  5
61
62
63
64
65
66
67
68
69
70
['4', ' 3', ' 5', ' 7', ' 2', ' 1', ' 2', ' 2', ' 1', ' 5']
61 4
62  3
63  5
64  7
65  2
66  1
67  2
68  2
69  1
70  5
71
72
73
74
75
76
77
78
79
80
['2', ' 1', ' 5', ' 1', ' 4', ' 4', ' 4', ' 3', ' 5', ' 2']
71 2
72  1
73  5

In [18]:
df.to_csv("2023_skynews_chatgpt_results_02.csv", sep=';')

In [22]:
response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
        {"role": "system", "content": "Generate a high quality text about a provided topic which is the size of 2,500 words"},
        {"role": "user", "content": f"Title: Fun"}
        ],
        temperature=0,
        max_tokens=4000
    )
category = response.choices[0].message.content
print(response)
print(category)


KeyboardInterrupt: 

In [42]:
counter = 0
for index, row in df.iterrows():
    print(counter)
    title = row['Title']
    tags = row['Tags'][1:-1]

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
        {"role": "system", "content": "You will be provided with a title and one or multiple tags of a news article. Your task is to classify to which of the following topics the article belongs to. The topics to choose from are: 'politics', 'business and economy', 'environment', 'sports', 'entertainment and culture', 'science and technology' or 'health'. Return only one of the provided topics and keep the same format as they are provided."},
        {"role": "user", "content": f"Title: '{title}'; Tags: {tags}"}
        ],
        temperature=0,
        max_tokens=256
    )
    category = response.choices[0].message.content
    df.loc[index, ['Chat GPT']] = category
    sleep(5)
    counter += 1


df.to_csv("2022_skynews_chatgpt_results_03.csv", sep=';')

NameError: name 'response' is not defined